In [ ]:
!pip install --upgrade transformers
!pip install datasets
!pip install faiss-gpu
!pip install sentence-transformers
!pip install scikit-learn



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 54.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct" , device_map=0)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
import spacy
import re

# Load spaCy's pre-trained NER model
nlp = spacy.load("en_core_web_sm")
def preprocess_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])


# Load a pre-trained sentence-transformers model
# Rename the model to 'sentence_transformer_model' for clarity and to avoid conflict
sentence_transformer_model = SentenceTransformer('all-mpnet-base-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
import os

# Get the current working directory
current_directory = os.getcwd()

# Define the path to the corpus.json file relative to the current directory
file_path = os.path.join(current_directory, "corpus.json")

# Load the dataset using the relative path
dataset = load_dataset("json", data_files=file_path)

def extract_metadata(doc):
    return {
        "title": doc["title"],
        "author": doc.get("author", "Unknown"),
        "url": doc.get("url", "N/A"),
        "source": doc.get("source", "N/A"),
        "category": doc.get("category", "General"),
        "published_at": doc.get("published_at", "N/A"),
        "body": doc["body"]
    }

# Assuming your dataset is named 'train' after loading
corpus = [extract_metadata(doc) for doc in dataset['train']]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Function to encode the corpus
def encode_corpus(corpus):
    document_texts = [preprocess_text(doc['body']) for doc in corpus]

    embeddings = sentence_transformer_model.encode(document_texts, convert_to_tensor=False)
    return np.array(embeddings), document_texts

# Step 1: Encode the corpus
corpus_embeddings, document_texts = encode_corpus(corpus)

# Step 2: Build a FAISS index
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # Using L2 distance (can switch to cosine if needed)

# Add embeddings to the FAISS index
index.add(corpus_embeddings)

In [ ]:
import spacy
import re
import string

# Load spaCy's pre-trained NER model
nlp = spacy.load("en_core_web_sm")

# Predefined list of common news sources
KNOWN_SOURCES = [
    "CNBC", "Fortune", "TechCrunch", "Hacker News", "The Epoch Times", "World Business News",
    "Mashable", "The Sydney Morning Herald", "Cnbc | World Business News Leader", "Yardbarker",
    "Sporting News", "Essentially Sports", "The New York Times", "TalkSport", "The Guardian",
    "The Verge", "Polygon", "The Independent - Life and Style", "FOX News - Entertainment",
    "The Age", "The Roar | Sports Writers Blog", "Engadget", "CBSSports.com", "Globes English"," Israel Business Arena",
    "Eos: Earth And Space Science News", "Music Business Worldwide", "Sky Sports", "Business Line",
    "Financial Times", "Wired", "Seeking Alpha", "Revyuh Media", "Insidesport", "Science News For Students",
    "Advanced Science News", "Live Science: The Most Interesting Articles", "BBC News"
]

# List of common articles/prepositions to remove from topics
ARTICLES = ['the', 'a', 'an']

def remove_articles(text):
    """Remove leading articles from proper nouns."""
    words = text.split()
    if words[0].lower() in ARTICLES:
        return ' '.join(words[1:])
    return text

def preprocess_text(text):
    """Preprocess the text by removing all punctuation and extra spaces."""
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text.strip()

def extract_entities_for_comparison(query):
    # Preprocess the query to remove punctuation and clean up text
    query = preprocess_text(query)

    # Use spaCy to process the query
    doc = nlp(query)

    # Extract entities
    entities = {'article_names': [], 'topics': [], 'actions': []}

    # Extract named entities (like organization names, locations, etc.)
    for ent in doc.ents:
        # Only consider the entity as a source if it matches a known source name
        if ent.label_ == 'ORG' and ent.text.strip() in KNOWN_SOURCES:
            entities['article_names'].append(ent.text.strip())
        elif ent.label_ == 'ORG' or ent.label_ == 'PERSON':
            # Treat as general organizations/people that may relate to topics
            entities['topics'].append(ent.text.strip())
        elif ent.label_ == 'GPE':  # Geopolitical entities (like cities, countries)
            entities['topics'].append(ent.text.strip())

    # Custom pattern matching for source names in case NER misses some
    source_pattern = r"(" + "|".join(re.escape(source) for source in KNOWN_SOURCES) + ")"
    found_sources = re.findall(source_pattern, query, re.IGNORECASE)
    entities['article_names'].extend(found_sources)

    # Remove duplicates and clean up partial matches
    entities['article_names'] = list(set(entities['article_names']))

    # Custom pattern matching for topics (like "software companies", "payment models", etc.)
    topic_keywords = ["software companies", "payment models", "subscription models",'guides',"purchasing", "home sales prices", "revenues", "defensive performance"]
    for keyword in topic_keywords:
        if keyword.lower() in query.lower():
            entities['topics'].append(keyword)

    # Extract specific comparison aspects using simple patterns
    comparison_keywords = ["increase in revenue", "decrease", "improve", "worse", "better", "compared to", "versus"]
    for keyword in comparison_keywords:
        if keyword.lower() in query.lower():
            entities['actions'].append(keyword)

    # Step 3: Add proper noun pairs to topics to avoid isolated words
    proper_nouns = []
    current_chunk = []
    for token in doc:
        # Check for proper nouns (tokens with POS tag 'PROPN')
        if token.pos_ == 'PROPN':
            current_chunk.append(token.text)
        else:
            if current_chunk:
                proper_nouns.append(" ".join(current_chunk))
                current_chunk = []

    # Add last chunk if any
    if current_chunk:
        proper_nouns.append(" ".join(current_chunk))

    # Filter out smaller, isolated proper nouns and add valid chunks to topics
    for chunk in proper_nouns:
        if len(chunk.split()) > 1:  # Only keep noun phrases with more than 1 word
            entities['topics'].append(chunk)

    # Remove duplicates from topics
    entities['topics'] = list(set(entities['topics']))

    # Step 4: Remove any article names from the topics
    entities['topics'] = [topic for topic in entities['topics'] if topic not in entities['article_names']]

    # Step 5: Remove leading articles from topics and ensure proper nouns appear once
    entities['topics'] = list(set(remove_articles(topic) for topic in entities['topics']))

    return entities

# Sample query for testing
query="Does the Sporting News article claim that Caesars Sportsbook offers a cash-out option for early bet settlement, while the CBSSports.com and the second Sporting News article both focus on Caesars Sportsbook providing a welcome bonus offer for new customers?"

# Test the extraction with the sample query
entities = extract_entities_for_comparison(query)

print("Extracted Entities:")
print(entities)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Extracted Entities:
{'article_names': ['Sporting News'], 'topics': ['Caesars Sportsbook', 'CBSSportscom'], 'actions': []}


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re



# Define the function for improved entity extraction using Llama model
def extract_entities_with_llama(query):
    # Step 1: Improved prompt with clear instructions for Llama model
    prompt = (
        f"Extract the following entities from the text: \n"
        f"1. Article names (news sources like TechCrunch, CNBC, Fortune, etc.)\n"
        f"2. Topics (relevant subjects like software companies, revenue, home sales prices, etc.)\n"
        f"3. Actions (comparison words like increase, decrease, improve, worse, etc.)\n\n"
        f"Text: \"{query}\"\n\n"
        f"Extracted Entities (please list all articles, topics, and actions found):\n"
        f"Article Names: \n"
        f"Topics: \n"
        f"Actions: \n"
    )

    # Step 2: Tokenize the input query
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Step 3: Generate text using the model
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=400,  # Increase max_length to allow for detailed responses
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9
        )

    # Step 4: Decode the output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)


    # Step 5: Post-process to extract the entities using regex
    # Look for the sections for each category
    article_names_match = re.search(r"Article Names:\s*(.*)", generated_text)
    topics_match = re.search(r"Topics:\s*(.*)", generated_text)
    actions_match = re.search(r"Actions:\s*(.*)", generated_text)


    # Extract the text after the labels and clean up
    article_names = article_names_match.group(1).strip() if article_names_match else ""
    topics = topics_match.group(1).strip() if topics_match else ""
    actions = actions_match.group(1).strip() if actions_match else ""

    # Split by commas and clean up
    extracted_entities = {
        "article_names": [name.strip() for name in article_names.split(",")] if article_names else [],
        "topics": [topic.strip() for topic in topics.split(",")] if topics else [],
        "actions": [action.strip() for action in actions.split(",")] if actions else []
    }

    return extracted_entities,generated_text

# Example usage

# print(entities)


In [ ]:
def extract_entities_from_output(query):

    entities,output_text= extract_entities_with_llama(query)
    lines = output_text.split('\n')

    article_names = []
    topics = []
    actions = []

    current_category = None

    for line in lines:
        line = line.strip()
        if line.startswith("Article Names:"):
            current_category = "article_names"
        elif line.startswith("Topics:"):
            current_category = "topics"
        elif line.startswith("Actions:"):
            current_category = "actions"
        elif line and current_category:
            # Remove the number and period at the start of the line, if present
            item = line.split('. ', 1)[-1].strip()
            if current_category == "article_names":
                article_names.append(item)
            elif current_category == "topics":
                topics.append(item)
            elif current_category == "actions":
                actions.append(item)

    return {
        "article_names": article_names,
        "topics": topics,
        "actions": actions
    }

query = "Does the Sporting News article claim that Caesars Sportsbook offers a cash-out option for early bet settlement, while the CBSSports.com and the second Sporting News article both focus on Caesars Sportsbook providing a welcome bonus offer for new customers?"

result = extract_entities_from_output(query)
print("Article Names:", result["article_names"])
print("Topics:", result["topics"])
print("Actions:", result["actions"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Article Names: ['Sporting News', 'CBSSports.com', 'Fortune', 'CNBC', 'TechCrunch']
Topics: ['Cash-out Option', 'Welcome Bonus', 'Early Bet Settlement', 'New Customers', 'Software Companies', 'Revenue', 'Home Sales Prices', 'Actions']
Actions: ['(Your response should be not exceed 2 lines.)', 'Sporting News', 'Caesars Sportsbook', 'CBSSports.com', 'Welcome Bonus', 'Cash-out Option', 'New Customers', 'Early Bet Settlement', 'Revenue', 'Home Sales Prices', 'Increase', 'Decrease', 'Improve', 'Worse', 'Fortune', 'CNBC', 'TechCrunch', 'Fortune', 'Software Companies', 'Software', 'Settlement', 'Offer', 'Please let me know if you need any further assistance.', 'I will revise the response to exclude those entities.', 'Revised Extracted Entities:']


In [ ]:
import faiss
import numpy as np

OPPOSITE_ACTIONS = {
    "increase": "decrease",
    "decrease": "increase",
    "better": "worse",
    "worse": "better",
    # Add other action-opposite pairs as needed
}

def expand_actions(actions):
    """
    Expand the list of actions by adding their opposites.
    """
    expanded_actions = set(actions)  # Use a set to avoid duplicates
    for action in actions:
        if action in OPPOSITE_ACTIONS:
            expanded_actions.add(OPPOSITE_ACTIONS[action])
    return list(expanded_actions)

def normalize_string(text):
    """
    Normalize strings for comparison by lowercasing and stripping extra spaces.
    """
    return text.lower().strip()



In [ ]:
def retrieve_documents_by_keywords(query, corpus, extract_entities_fn):
    """
    Retrieve documents based on the extracted keywords from the query.

    Parameters:
    - query: The query string to search for.
    - corpus: The original corpus containing documents.
    - extract_entities_fn: Function to extract entities from the query.

    Returns:
    - List of documents that match the extracted keywords (sources, topics, actions).
    """
    # Step 1: Extract entities from the query using the provided function
    entities = extract_entities_fn(query)
    print(entities)

    # Step 2: Check if any entities were extracted
    if not any([entities['article_names'], entities['topics'], entities['actions']]):
        # If no entities were extracted, fallback to using the full query for document retrieval
        return [doc for doc in corpus if query.lower() in normalize_string(doc.get('body', '')).lower()]

    # Step 3: Expand actions to include their opposites
    expanded_actions = expand_actions(entities['actions']) if entities['actions'] else []

    # Step 4: Normalize article names, topics, and actions for consistent comparison
    normalized_article_names = [normalize_string(name) for name in entities['article_names']] if entities['article_names'] else []
    normalized_topics = [normalize_string(topic) for topic in entities['topics']] if entities['topics'] else []
    normalized_actions = [normalize_string(action) for action in expanded_actions] if expanded_actions else []

    # Step 5: Filter documents based on available keywords (sources, topics, actions)
    matching_docs = []

    for doc in corpus:
        doc_title = normalize_string(doc.get('source', ''))
        doc_text = normalize_string(doc.get('body', ''))

        # Check if the document matches the source (article name) only if article names exist
        matches_source = any(article_name in doc_title for article_name in normalized_article_names) if normalized_article_names else True

        # Check if the document matches the topics only if topics exist
        matches_topic = any(topic in doc_text for topic in normalized_topics) if normalized_topics else True

        # Check if the document matches the actions only if actions exist
        #matches_action = any(action in doc_text for action in normalized_actions) if normalized_actions else True

        # If the document matches source, topic, and action, it's a valid match
        if matches_source and matches_topic:
            matching_docs.append(doc)

    return matching_docs


In [ ]:
def finretrieve_documents_faiss(query,retdocs, top_k=5):
    # Step 1: Encode the query into an embedding
    # Step 1: Encode the corpus
    retdocs_embeddings, document_texts = encode_corpus(retdocs)

    # Step 2: Build a FAISS index
    retdimension = retdocs_embeddings.shape[1]
    retindex = faiss.IndexFlatL2(retdimension)  # Using L2 distance (can switch to cosine if needed)

    # Add embeddings to the FAISS index
    retindex.add(retdocs_embeddings)

    # Use 'sentence_transformer_model' for encoding
    query_embedding = sentence_transformer_model.encode([preprocess_text(query)], convert_to_tensor=False)

    # Step 2: Search FAISS index for top-k similar documents
    distances, indices = retindex.search(np.array(query_embedding), top_k)

    # Step 3: Retrieve the top-k documents based on the indices
    retrieved_docs = [retdocs[i] for i in indices[0]]
    return retrieved_docs

# query="Does the Sporting News article claim that Caesars Sportsbook offers a cash-out option for early bet settlement, while the CBSSports.com and the second Sporting News article both focus on Caesars Sportsbook providing a welcome bonus offer for new customers?"
# # Step 1: Use the new retrieval function
# finalll = finretrieve_documents_faiss(query,retrieved_documents)

# # Step 2: Display the retrieved documents
# print("Retrieved Documents:")
# for doc in finalll:
#     print(f"source: {doc['source']}")
#     print(f"title: {doc['title']}")


Retrieved Documents:
source: Sporting News
title: Best sportsbook bonus offers for NFL Monday Night Football Eagles vs. Seahawks: Claim over $5,000 in bonuses from Bet365, BetMGM, BetRivers, Caesars Sportsbook, DraftKings and FanDuel 
source: Sporting News
title: NFL Best Bets Week 9: Colts clobber Panthers, Dolphins drop Chiefs in Germany
source: Sporting News
title: Line Shopping in Sports Betting - How to Find the Best Odds
source: Sporting News
title: Best Golf Betting Sites and Apps - Top Sportsbooks for Golf 2023
source: Sporting News
title: Vermont Sportsbook Promos and Sports Betting Launch News


In [ ]:



def retrieve_documents_faiss(query, top_k=4):
    # Step 1: Encode the query into an embedding
    # Use 'sentence_transformer_model' for encoding
    query_embedding = sentence_transformer_model.encode([query], convert_to_tensor=False)

    # Step 2: Search FAISS index for top-k similar documents
    distances, indices = index.search(np.array(query_embedding), top_k)

    # Step 3: Retrieve the top-k documents based on the indices
    retrieved_docs = [corpus[i] for i in indices[0]] # Access the global 'corpus' here
    return retrieved_docs

# query="Do 'The Verge' and 'Engadget' articles both suggest that 'Consumers' have guides or opportunities to make better purchasing decisions, while 'TechCrunch' discusses 'Consumers' desire for a new model in a different sector?"

# # Call the function with only the query and optional top_k
# finalll = retrieve_documents_faiss(query) # top_k defaults to 4 if not provided

# # Step 2: Display the retrieved documents
# print("Retrieved Documents:")
# for doc in finalll:
#     print(f"source: {doc['source']}")
#     print(f"title: {doc['title']}")

In [ ]:
def main_query_answering(query, documents, max_doc_length=400, max_tokens=100):
    """
    Answer the main query using Meta-Llama-3.2-1B and the retrieved documents.
    """
    # Prepare the prompt using the main query and the relevant shortened documents
    # Assuming 'body' key holds the document text in retrieved_documents
    short_docs = [doc.get('body', '')[:max_doc_length] for doc in documents]
    prompt = f"Answer the following question and only output the answer based on the provided documents and answer in only yes or no:\nDocuments: {short_docs}\nQuery: {query}\nAnswer:"

    # Tokenize input for LLM
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate answer for the main query with reduced token output
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, temperature=0.5)

    # Decode and extract answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    return answer

# Example main query
# query = "Does 'The New York Times' article attribute the success of the Buffalo Bills' defense to the contributions of Jordan Poyer, while the 'Sporting News' article suggests that the Baltimore Ravens' defense needs to improve before their game against the Cincinnati Bengals?"

# # Shortened documents for faster processing
# docs = [doc['body'][:200] for doc in retrieved_documents]

# # Answer the main query using the shortened documents
# final_answer = main_query_answering(query, finalll)

# # Display the final answer
# print("Final Answer:", final_answer)


In [ ]:
def main_query_answering2(query, documents, max_doc_length=500, max_tokens=150):
    """
    Answer the main query using Meta-Llama-3.2-1B and the retrieved documents.
    """
    # Prepare the prompt using the main query and the relevant shortened documents
    # Assuming 'body' key holds the document text in retrieved_documents
    short_docs = [doc.get('body', '')[:max_doc_length] for doc in documents]
    prompt = f"Answer the following question and only output the answer based on the provided documents and answer in only yes or no and provide explanation in another line:\nDocuments: {short_docs}\nQuery: {query}\nAnswer:"

    # Tokenize input for LLM
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate answer for the main query with reduced token output
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, temperature=0.5)

    # Decode and extract answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    return answer

# Example main query
# query = "Does 'The New York Times' article attribute the success of the Buffalo Bills' defense to the contributions of Jordan Poyer, while the 'Sporting News' article suggests that the Baltimore Ravens' defense needs to improve before their game against the Cincinnati Bengals?"

# # Shortened documents for faster processing
# docs = [doc['body'][:200] for doc in retrieved_documents]

# # Answer the main query using the shortened documents
# final_answer = main_query_answering(query, finalll)

# # Display the final answer
# print("Final Answer:", final_answer)


In [ ]:
def rag_pipeline(query, corpus):
    # Step 1: Retrieve documents based on the query
    retrieved_docs = retrieve_documents_by_keywords(query, corpus,extract_entities_for_comparison)

    # Step 2: Use FAISS to retrieve top-k documents based on query
    faiss_docs = finretrieve_documents_faiss(query, retrieved_docs)
    for i in faiss_docs:
        print(i["title"])

    # Step 3: Answer the main query based on the retrieved documents
    final_answer = main_query_answering2(query, faiss_docs)

    return final_answer

query="Does the TechCrunch article suggest that Sam Altman is involved in a new venture, while the Fortune article discusses the roles of Matt Huang and Gary Wang in the context of venture capital and FTX, without indicating any new venture involvement for them?"

final_answer = rag_pipeline(query, corpus)
print("Final Answer:", final_answer)

{'article_names': ['Fortune', 'TechCrunch'], 'topics': ['Matt Huang', 'Gary Wang', 'Sam Altman'], 'actions': []}


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


A timeline of Sam Altman’s firing from OpenAI — and the fallout
SBF’s trial starts soon, but how did he — and FTX — get here?
Is Sam Bankman-Fried a bad ‘man’ or a good ‘boy’? Lawyers swap opening statements before first witnesses take the stand
WTF is going on at OpenAI? We have theories
Sam Altman ousted as OpenAI’s CEO
Final Answer: Answer the following question and only output the answer based on the provided documents and answer in only yes or no and provide explanation in another line:
Documents: ['In a dramatic turn of events late Friday, ex-Y Combinator president Sam Altman was fired as CEO of AI startup OpenAI, the company behind viral AI hits like ChatGPT, GPT-4 and DALL-E 3, by OpenAI’s board of directors. Then, the company’s longtime president and co-founder, Greg Brockman, resigned — as did three senior OpenAI researchers. And the fallout continues.\n\nTip TechCrunch\n\nDo you work at OpenAI and know more about Sam Altman’s departure?\n\nDo you work at OpenAI and know more